In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path
from scripts.cross_match_scripts import my_scaler_forward
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', 500)
import seaborn as sns
import numpy as np
%matplotlib inline

# Catalogs loading and preprocessing

In [3]:
photo_cat = pd.read_pickle(data_path+'csc_desi_r50_gaia_traning.gz_pkl', compression='gzip')
x_ray_flux_lim = 3e-15
photo_cat = photo_cat.query('(flux_csc_05_2 > @x_ray_flux_lim) | is_counterpart == False')
photo_cat = photo_cat.replace([np.inf, -np.inf], np.nan)
photo_cat.head(5)

,name_csc,ra_csc,dec_csc,sep_csc,release,brickid,objid,brick_primary,type,ra,dec,ra_ivar,dec_ivar,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,pmra,pmdec,parallax,pmra_ivar,pmdec_ivar,parallax_ivar,flux_csc_05_2,r_98_csc,mag_g,mag_r,mag_z,mag_w1,mag_w2,mag_w3,mag_w4,desi_id,r_false_003_dens_10_50,sep_less_r_false,is_counterpart
122,2CXO J000001.7+003824,0.007351,0.640170,0.510857,9010,334688,289,True,b'PSF',0.007209,0.640172,7.968032e+10,7.686111e+10,0.446684,0.791833,0.833604,2.084917,-1.714462,-19.942390,-1472.021851,0.0,0.0,0.0,0.0,0.0,0.0,4.925396e-15,1.598058,23.374999,22.753416,22.697600,21.702278,NaN,NaN,NaN,9010_334688_289,0.783775,1,True
706,2CXO J000002.4+004444,0.010261,0.745756,0.168650,9010,334688,428,True,b'DEV',0.010231,0.745719,4.837769e+09,2.799387e+09,0.409332,0.494572,1.014339,4.271025,2.108738,-69.947258,-2105.617432,0.0,0.0,0.0,0.0,0.0,0.0,6.223680e-15,1.187993,23.469812,23.264425,22.484542,20.923670,21.689943,NaN,NaN,9010_334688_428,0.747015,1,True
1151,2CXO J000004.2+004038,0.017509,0.677271,0.340550,9010,334688,728,True,b'DEV',0.017430,0.677220,1.214703e+10,1.709058e+10,0.834075,1.629502,2.029589,12.394610,8.341883,151.319443,-9200.514648,0.0,0.0,0.0,0.0,0.0,0.0,3.489255e-15,1.210327,22.696987,21.969863,21.731480,19.766918,20.196840,17.050263,NaN,9010_334688_728,0.758688,1,True
1528,2CXO J000004.5-552604,0.018973,-55.434463,0.302039,9010,58166,392,True,b'PSF',0.018886,-55.434395,5.386807e+11,8.053757e+11,2.826036,3.806152,4.085902,24.098080,35.776527,-103.290527,7386.880371,0.0,0.0,0.0,0.0,0.0,0.0,1.030507e-14,2.599839,21.372056,21.048785,20.971780,19.045044,18.616005,NaN,12.828847,9010_58166_392,0.850754,1,True
1634,2CXO J000004.7+004654,0.019850,0.781780,0.341217,9010,334688,828,True,b'SER',0.019784,0.781712,4.079795e+13,4.402594e+13,257.798218,611.787598,1161.549072,1052.845337,677.435303,1074.324219,1625.962158,0.0,0.0,0.0,0.0,0.0,0.0,3.925906e-15,1.477375,16.471800,15.533498,14.837406,14.944089,15.422830,14.922162,14.472224,9010_334688_828,0.767811,1,True


In [4]:
print('fraction of missed measurements per is_counterpart per filter, in per cent:')
photo_cat.groupby('is_counterpart')[['mag_g',	'mag_r',	'mag_z',	'mag_w1',	'mag_w2',	'mag_w3',	'mag_w4']].agg(lambda x: np.round(100*np.mean(np.
isnan(x)),1))

fraction of missed measurements per is_counterpart per filter, in per cent:


,mag_g,mag_r,mag_z,mag_w1,mag_w2,mag_w3,mag_w4
is_counterpart,,,,,,,
False,2.9,1.8,2.3,27.2,36.7,51.5,51.3
True,1.5,1.2,0.8,2.4,2.7,20.5,37.1


In [5]:
features_cols = ['mag_g','mag_r','mag_z'] 
features_cols += ['col_gr', 'col_rz', 'col_gz']

photo_cat['col_gr'] = photo_cat['mag_g'] - photo_cat['mag_r']
photo_cat['col_rz'] = photo_cat['mag_r'] - photo_cat['mag_z']
photo_cat['col_gz'] = photo_cat['mag_g'] - photo_cat['mag_z']


features_cols += ['mag_w1', 'mag_w2']
features_cols += ['col_zw1', 'col_rw2', 'col_w1w2']
photo_cat['col_zw1'] = photo_cat['mag_z'] - photo_cat['mag_w1']
photo_cat['col_rw2'] = photo_cat['mag_r'] - photo_cat['mag_w2']
photo_cat['col_w1w2'] = photo_cat['mag_w1'] - photo_cat['mag_w2']


features_cols += ['mag_w3', 'mag_w4']
features_cols += ['col_zw3', 'col_rw4', 'col_w3w4']
photo_cat['col_zw3'] = photo_cat['mag_z'] - photo_cat['mag_w3']
photo_cat['col_rw4'] = photo_cat['mag_r'] - photo_cat['mag_w4']
photo_cat['col_w3w4'] = photo_cat['mag_w3'] - photo_cat['mag_w4']


target_col = ['is_counterpart']

photo_cat = photo_cat[features_cols+target_col+['flux_csc_05_2']]

#photo_cat= photo_cat.dropna(subset = features_cols, how = 'any')

print(photo_cat.is_counterpart.value_counts())
photo_cat = photo_cat.sample(frac=1)
photo_cat.reset_index(inplace=True, drop = True)

photo_cat_scaled = my_scaler_forward(photo_cat)

photo_cat_scaled.to_pickle('./model/train_test_validation_data.pkl')

photo_cat.head(15)


False    326948
True      39102
Name: is_counterpart, dtype: int64


,mag_g,mag_r,mag_z,col_gr,col_rz,col_gz,mag_w1,mag_w2,col_zw1,col_rw2,col_w1w2,mag_w3,mag_w4,col_zw3,col_rw4,col_w3w4,is_counterpart,flux_csc_05_2
0,23.825863,23.024691,22.198704,0.801172,0.825987,1.627159,21.766128,23.489187,0.432576,-0.464496,-1.723059,17.506463,15.400886,4.692240,7.623805,2.105577,False,NaN
1,25.197764,24.531854,NaN,0.665910,NaN,NaN,NaN,23.183121,NaN,1.348733,NaN,21.769864,15.047691,NaN,9.484163,6.722173,False,NaN
2,23.800436,23.088762,22.561853,0.711674,0.526909,1.238583,22.338912,20.994904,0.222941,2.093858,1.344008,NaN,NaN,NaN,NaN,NaN,False,NaN
3,19.223836,18.901496,18.540409,0.322341,0.361087,0.683428,17.572601,17.395139,0.967808,1.506357,0.177462,16.102180,14.276251,2.438228,4.625245,1.825929,True,6.501821e-14
4,22.766075,21.463315,20.724821,1.302760,0.738494,2.041254,20.376960,20.558350,0.347861,0.904966,-0.181389,20.840560,14.897582,-0.115739,6.565733,5.942978,False,NaN
5,16.643986,16.317815,16.049507,0.326171,0.268307,0.594478,15.982794,15.599880,0.066714,0.717934,0.382913,14.315371,13.539863,1.734136,2.777951,0.775508,True,1.309936e-13
6,24.755625,25.185218,24.538131,-0.429592,0.647087,0.217494,NaN,21.594274,NaN,3.590944,NaN,NaN,15.381072,NaN,9.804146,NaN,False,NaN
7,18.601310,18.150714,17.970316,0.450596,0.180398,0.630995,19.310084,20.006126,-1.339769,-1.855412,-0.696042,NaN,18.624482,NaN,-0.473767,NaN,False,NaN
8,23.568783,22.972773,21.651276,0.596010,1.321497,1.917507,20.892816,20.881868,0.758460,2.090906,0.010948,NaN,14.997319,NaN,7.975455,NaN,False,NaN
9,21.278401,21.047425,21.194186,0.230977,-0.146762,0.084215,19.915146,19.768572,1.279041,1.278853,0.146574,19.681316,15.928540,1.512870,5.118885,3.752777,True,1.281387e-14
